In [1]:
ls

 Volume in drive C is Acer
 Volume Serial Number is 3818-7542

 Directory of C:\Users\phvpa\OneDrive - Amrita Vishwa Vidyapeetham\Amrita-Mtech-CEN\ProjectWork\MTech Thesis\Denoise in Code mix data\Experiments\Malyalam\MuRIL\Lakshmi_MuRIL

22-10-2021  16:03    <DIR>          .
22-10-2021  16:01    <DIR>          ..
22-10-2021  16:03    <DIR>          .ipynb_checkpoints
22-10-2021  15:58        50,122,126 drive-download-20211022T102758Z-001.zip
22-10-2021  13:29            29,302 MuRIL_Denoising.ipynb
22-10-2021  16:01        48,808,064 Muril_X_train.npy
22-10-2021  16:01         5,425,280 Muril_X_val.npy
22-10-2021  12:15               360 WR_LS_Denoising.py
22-10-2021  03:50           127,232 Y_train.npy
22-10-2021  03:50            14,256 Y_val.npy
               7 File(s)    104,526,620 bytes
               3 Dir(s)  335,302,082,560 bytes free


In [2]:
!nvidia-smi

Fri Oct 22 16:04:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   55C    P0    N/A /  N/A |    121MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from WR_LS_Denoising import WR_LS_Denoise
import numpy as np
import cupy as cp

In [4]:
X_train = cp.load('MuRIL_X_train.npy')
X_val = cp.load('MuRIL_X_val.npy')
Y_train = cp.load('Y_train.npy')
Y_val = cp.load('Y_val.npy')

In [5]:
import time

In [6]:
start = time.ctime(time.time())
print(start)
for i in range(len(X_train)):
    Denoised =  WR_LS_Denoise(X_train[i])
    X_train[i] = X_train[i]
    if(i%1000 == 0):
        print(i, time.ctime(time.time()))

Fri Oct 22 16:04:28 2021
0 Fri Oct 22 16:04:30 2021
1000 Fri Oct 22 16:05:18 2021
2000 Fri Oct 22 16:06:06 2021
3000 Fri Oct 22 16:06:56 2021
4000 Fri Oct 22 16:07:47 2021
5000 Fri Oct 22 16:08:37 2021
6000 Fri Oct 22 16:09:28 2021
7000 Fri Oct 22 16:10:18 2021
8000 Fri Oct 22 16:11:07 2021
9000 Fri Oct 22 16:11:57 2021
10000 Fri Oct 22 16:12:47 2021
11000 Fri Oct 22 16:13:37 2021
12000 Fri Oct 22 16:14:28 2021
13000 Fri Oct 22 16:15:18 2021
14000 Fri Oct 22 16:16:07 2021
15000 Fri Oct 22 16:16:57 2021


In [7]:
start = time.ctime(time.time())
print(start)
for i in range(len(X_val)):
    Denoised =  WR_LS_Denoise(X_val[i])
    X_val[i] = X_val[i]
    if(i%1000 == 0):
        print(i, time.ctime(time.time()))

Fri Oct 22 16:17:51 2021
0 Fri Oct 22 16:17:51 2021
1000 Fri Oct 22 16:18:41 2021


In [8]:
WR_LS_Denoised_X_train = cp.save('WR_LS_Denoised_MuRIL_X_train.npy',X_train)
WR_LS_Denoised_X_val = cp.save('WR_LS_Denoised_MuRIL_X_val.npy',X_val)

In [9]:
X_train = np.load('WR_LS_Denoised_MuRIL_X_train.npy')
X_val = np.load('WR_LS_Denoised_MuRIL_X_val.npy')

In [10]:
Y_train = np.load('Y_train.npy')
Y_val = np.load('Y_val.npy')

# SVM

In [12]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

In [13]:
svc = SVC(decision_function_shape='ovo')
svc.fit(X_train,Y_train)

SVC(decision_function_shape='ovo')

In [14]:
pred_svc = svc.predict(X_val)

In [15]:
confusion_matrix(Y_val,pred_svc)

array([[  0,   0, 102,   0,   0],
       [  0,   0, 237,   0,   0],
       [  0,   0, 704,   0,   2],
       [  0,   0, 141,   0,   0],
       [  0,   0, 580,   0,   0]], dtype=int64)

In [16]:
print(classification_report(Y_val,pred_svc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       102
           1       0.00      0.00      0.00       237
           2       0.40      1.00      0.57       706
           3       0.00      0.00      0.00       141
           4       0.00      0.00      0.00       580

    accuracy                           0.40      1766
   macro avg       0.08      0.20      0.11      1766
weighted avg       0.16      0.40      0.23      1766



C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# DecisionTree

In [17]:
from sklearn.tree import DecisionTreeClassifier

In [18]:
Dtree = DecisionTreeClassifier(criterion='entropy', random_state=10)

In [19]:
Dtree.fit(X_train,Y_train)

DecisionTreeClassifier(criterion='entropy', random_state=10)

In [20]:
pred_Dtree = Dtree.predict(X_val)

In [21]:
confusion_matrix(Y_val,pred_Dtree)

array([[ 17,  14,  44,   3,  24],
       [ 16,  61,  87,   8,  65],
       [ 39,  96, 352,  28, 191],
       [  1,   9,  34,  69,  28],
       [ 33,  82, 209,  18, 238]], dtype=int64)

In [22]:
print(classification_report(Y_val,pred_Dtree))

              precision    recall  f1-score   support

           0       0.16      0.17      0.16       102
           1       0.23      0.26      0.24       237
           2       0.48      0.50      0.49       706
           3       0.55      0.49      0.52       141
           4       0.44      0.41      0.42       580

    accuracy                           0.42      1766
   macro avg       0.37      0.36      0.37      1766
weighted avg       0.42      0.42      0.42      1766



# Random Forest

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
RFC = RandomForestClassifier()
RFC.fit(X_train,Y_train)

RandomForestClassifier()

In [25]:
pred_RFC = RFC.predict(X_val)

In [26]:
confusion_matrix(Y_val,pred_RFC)

array([[  6,   0,  67,   1,  28],
       [  1,  28, 147,   1,  60],
       [  1,   2, 584,   5, 114],
       [  0,   0,  65,  62,  14],
       [  1,   4, 281,   3, 291]], dtype=int64)

In [27]:
print(classification_report(Y_val,pred_RFC))

              precision    recall  f1-score   support

           0       0.67      0.06      0.11       102
           1       0.82      0.12      0.21       237
           2       0.51      0.83      0.63       706
           3       0.86      0.44      0.58       141
           4       0.57      0.50      0.54       580

    accuracy                           0.55      1766
   macro avg       0.69      0.39      0.41      1766
weighted avg       0.61      0.55      0.51      1766



# Orginal

In [28]:
X_train = np.load('MuRIL_X_train.npy')
X_val = np.load('MuRIL_X_val.npy')
Y_train = np.load('Y_train.npy')
Y_val = np.load('Y_val.npy')

In [29]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

In [30]:
svc = SVC(decision_function_shape='ovo')
svc.fit(X_train,Y_train)

SVC(decision_function_shape='ovo')

In [31]:
pred_svc = svc.predict(X_val)

In [32]:
confusion_matrix(Y_val,pred_svc)

array([[  0,   0, 102,   0,   0],
       [  0,   0, 237,   0,   0],
       [  0,   0, 704,   0,   2],
       [  0,   0, 141,   0,   0],
       [  0,   0, 580,   0,   0]], dtype=int64)

In [33]:
print(classification_report(Y_val,pred_svc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       102
           1       0.00      0.00      0.00       237
           2       0.40      1.00      0.57       706
           3       0.00      0.00      0.00       141
           4       0.00      0.00      0.00       580

    accuracy                           0.40      1766
   macro avg       0.08      0.20      0.11      1766
weighted avg       0.16      0.40      0.23      1766



C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
Dtree = DecisionTreeClassifier(criterion='entropy', random_state=10)

In [36]:
Dtree.fit(X_train,Y_train)

DecisionTreeClassifier(criterion='entropy', random_state=10)

In [37]:
pred_Dtree = Dtree.predict(X_val)

In [38]:
confusion_matrix(Y_val,pred_Dtree)

array([[ 17,  14,  44,   3,  24],
       [ 16,  61,  87,   8,  65],
       [ 39,  96, 352,  28, 191],
       [  1,   9,  34,  69,  28],
       [ 33,  82, 209,  18, 238]], dtype=int64)

In [39]:
print(classification_report(Y_val,pred_Dtree))

              precision    recall  f1-score   support

           0       0.16      0.17      0.16       102
           1       0.23      0.26      0.24       237
           2       0.48      0.50      0.49       706
           3       0.55      0.49      0.52       141
           4       0.44      0.41      0.42       580

    accuracy                           0.42      1766
   macro avg       0.37      0.36      0.37      1766
weighted avg       0.42      0.42      0.42      1766



In [40]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
RFC = RandomForestClassifier()
RFC.fit(X_train,Y_train)

RandomForestClassifier()

In [42]:
pred_RFC = RFC.predict(X_val)

In [43]:
print(classification_report(Y_val,pred_RFC))

              precision    recall  f1-score   support

           0       0.75      0.06      0.11       102
           1       0.87      0.11      0.20       237
           2       0.53      0.85      0.65       706
           3       0.84      0.46      0.60       141
           4       0.59      0.52      0.56       580

    accuracy                           0.57      1766
   macro avg       0.72      0.40      0.42      1766
weighted avg       0.63      0.57      0.52      1766

